In [1]:
import torch
import gym
import gym_super_mario_bros
from gym import RewardWrapper
from gym_super_mario_bros import make

from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT
import cv2
import numpy as np  # ✅ Ensure NumPy is imported
import matplotlib.pyplot as plt

from gym.wrappers import FrameStack, GrayScaleObservation, ResizeObservation

from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

from matplotlib import pyplot as plt 

#import alldependencies
import os 
from stable_baselines3 import PPO
#for saving the models every xxx amount of steps 
from stable_baselines3.common.callbacks import BaseCallback

In [2]:


class NoSeedResetWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        kwargs.pop("seed", None)     # ✅ Remove 'seed' argument
        kwargs.pop("options", None)  # ✅ Remove 'options' argument
        return self.env.reset(**kwargs)
class CustomRewardWrapper(RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.previous_x = 0  # Guardar la posición anterior de Mario

    def step(self, action):
        """
        Llama al método `step()` del entorno y modifica la recompensa antes de retornarla.
        """
        observation, reward, done, truncated, info = self.env.step(action)  # Extraer `info` correctamente
        
        current_x = info['x_pos']  # Posición de Mario en el eje X
        current_time = info['time']  # Tiempo restante en el juego
        
        # Penalización por paso del tiempo (Reducida para que no castigue demasiado)
        time_penalty = -0.05  

        # Incentivar avance en la pantalla (Aumento de la recompensa por avanzar)
        progress_reward = (current_x - self.previous_x) * 0.2  
        
        # Bonificación adicional por mantener una velocidad alta
        if (current_x - self.previous_x) > 3:  # Si avanza más de 3 unidades en un solo paso
            progress_reward += 0.1  

        self.previous_x = current_x  # Actualizar la posición anterior
        
        # Penalizar quedarse quieto
        if progress_reward <= 0:
            progress_reward -= 0.01  # Penalización leve si no avanza
        
        # Penalización fuerte si Mario pierde todas sus vidas
        if info['life'] == 0:
            progress_reward -= 50
        
        # Modificar la recompensa final
        modified_reward = reward + progress_reward + time_penalty
        
        return observation, modified_reward, done, truncated, info  # Retornar los valores corregidos


In [3]:
#create base env 
env = gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode="human")

env = JoypadSpace(env, SIMPLE_MOVEMENT)

#actualice the reward system 
#env = CustomRewardWrapper(env)  # Aplicar el wrapper de recompensa

#RESIZES THE SHAPE FOR IT TO CONSUME LESS ENERGY
#env = ResizeObservation(env, shape=84)

#gray scale the env
env = GrayScaleObservation(env, keep_dim=True)
env = NoSeedResetWrapper(env)
#wrap inside dummyy
env = DummyVecEnv([lambda: env])
#stack

env = VecFrameStack(env, 4, channels_order='last')

C:\Users\santi\AppData\Roaming\Python\Python38\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
C:\Users\santi\AppData\Roaming\Python\Python38\site-packages\gym\envs\registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(
C:\Users\santi\AppData\Roaming\Python\Python38\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [4]:
#state = env.reset()
#state.shape
# viene en tuplet la informacion del env.reset  entonces algunas cosas no va a imprimir bien
#state, _ = env.reset()  # Extract only the state
#print(f"State Shape: {state.shape}")
# Reset the environment
state = env.reset()

print(f"State Type: {type(state)}")
print(f"State Shape: {getattr(state, 'shape', 'No Shape Attribute')}")
print(f"State Min: {state.min()}, State Max: {state.max()}")


# If using Gymnasium, state is a tuple (state, info), so extract the state
if isinstance(state, tuple):
    state = state[0]  # Extract the actual observation

# Print the shape of the stacked frames
print(f"State Shape: {state.shape}")


State Type: <class 'numpy.ndarray'>
State Shape: (1, 240, 256, 4)
State Min: 0, State Max: 252
State Shape: (1, 240, 256, 4)


In [5]:
model = PPO.load('./train/best_model_440000')
state = env.reset()

In [ ]:
state= env.reset()
while True:
    action, _ = model.predict(state)
    state, reward,done, info = env.step(action)
    env.render

C:\Users\santi\AppData\Roaming\Python\Python38\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
C:\Users\santi\AppData\Roaming\Python\Python38\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256
